In [ ]:
# pandas, matplotlib, seaborn csv 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

from IPython.display import set_matplotlib_formats

import warnings
warnings.filterwarnings(action='ignore')

#폰트가 선명하게 보이기 위해
set_matplotlib_formats('retina')

file_path = "C:/Users/j_er/Desktop/Salary.csv"


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import numpy as np

# 데이터 불러오기
data = pd.read_csv(file_path)

#Race, Senior 칼럼 제거
columns_to_remove = ['Race', 'Senior']

data = data.drop(columns=columns_to_remove)

# 'Front End Developer'를 'Front end Developer'로 변경
data['Job Title'] = data['Job Title'].replace('Front End Developer', 'Front end Developer')

# 제외할 직종 목록
exclude_titles = ['Copywriter', 'Data Entry Clerk', 'Director of Human Capital',
                  'Support Specialist', 'Technical Recruiter', 'Recruiter', 'Digital Marketing Manager', 'Digital Marketing Specialist']

# 제외할 직종 제거
data = data[~data['Job Title'].isin(exclude_titles)]

# IT 및 소프트웨어 관련 직종 필터링
it_sw_keywords = ['Software', 'Developer', 'Engineer', 'Data', 'System', 'IT', 'Tech', 'Programmer', 'Web', 'Network']
it_sw_related_jobs = data[data['Job Title'].str.contains('|'.join(it_sw_keywords), case=False)]

# 원-핫 인코딩 적용
categorical_columns = ['Job Title', 'Gender', 'Education Level', 'Country']
encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(it_sw_related_jobs[categorical_columns])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))
combined_data = pd.concat([it_sw_related_jobs.drop(categorical_columns, axis=1).reset_index(drop=True), encoded_df], axis=1)

# 랜덤 포레스트 모델 구축 및 평가
X = combined_data.drop('Salary', axis=1)
y = combined_data['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred))
r2_rf = r2_score(y_test, y_pred)


#랜덤 포레스트 평가지표
print("===랜덤포레스트===")
print(f"RMSE: {rmse_rf}")
print(f"R² score: {r2_rf}")
print("\n")


# 선형 회귀 모델 생성 및 학습
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 테스트 세트로 예측
y_pred = lr_model.predict(X_test)

# 선형회귀 평가
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred))
r2_linear = r2_score(y_test, y_pred)

# 결과 출력
print("===선형회귀===")
print(f"RMSE: {rmse_linear}")
print(f"R² score: {r2_linear}")

===랜덤포레스트===
RMSE: 10481.318579011153
R² score: 0.9559330180667114


===선형회귀===
RMSE: 25950.186012817667
R² score: 0.7298764709737828


In [ ]:
# 'combined_data' 데이터프레임의 모든 열 확인
print(combined_data.columns)

Index(['Age', 'Years of Experience', 'Salary', 'Job Title_Back end Developer',
       'Job Title_Chief Data Officer', 'Job Title_Chief Technology Officer',
       'Job Title_Data Analyst', 'Job Title_Data Engineer',
       'Job Title_Data Scientist', 'Job Title_Developer',
       'Job Title_Digital Content Producer',
       'Job Title_Director of Data Science',
       'Job Title_Director of Engineering', 'Job Title_Engineer',
       'Job Title_Front end Developer', 'Job Title_Full Stack Engineer',
       'Job Title_IT Consultant', 'Job Title_IT Manager',
       'Job Title_IT Project Manager', 'Job Title_IT Support',
       'Job Title_IT Support Specialist', 'Job Title_Network Engineer',
       'Job Title_Principal Engineer', 'Job Title_Project Engineer',
       'Job Title_Quality Assurance Analyst', 'Job Title_Software Architect',
       'Job Title_Software Developer', 'Job Title_Software Engineer',
       'Job Title_Software Engineer Manager', 'Job Title_Software Manager',
       'Job

In [ ]:
# 'Country' 관련 원-핫 인코딩된 열들 찾기
country_columns = [col for col in combined_data.columns if col.startswith('Country_')]

# 'Job Title' 관련 원-핫 인코딩된 열들 찾기
job_title_columns = [col for col in combined_data.columns if col.startswith('Job Title_')]

# 각 국적별 직종별 종사자 수 계산
country_job_title_counts = {}
for country_col in country_columns:
    country_data = combined_data[combined_data[country_col] == 1]
    job_counts = country_data[job_title_columns].sum()
    country_job_title_counts[country_col] = job_counts

# 결과 출력
for country, counts in country_job_title_counts.items():
    print(f"{country}:\n{counts.sort_values(ascending=False)}\n")

Country_Australia:
Job Title_Software Engineer               160.0
Job Title_Data Scientist                  106.0
Job Title_Data Analyst                     83.0
Job Title_Software Engineer Manager        74.0
Job Title_Project Engineer                 72.0
Job Title_Front end Developer              54.0
Job Title_Full Stack Engineer              54.0
Job Title_Software Developer               50.0
Job Title_Back end Developer               42.0
Job Title_Web Developer                    31.0
Job Title_Director of Data Science         12.0
Job Title_Data Engineer                     2.0
Job Title_Developer                         1.0
Job Title_IT Consultant                     1.0
Job Title_IT Manager                        1.0
Job Title_Technical Writer                  1.0
Job Title_Software Manager                  1.0
Job Title_Software Architect                1.0
Job Title_Engineer                          0.0
Job Title_Web Designer                      0.0
Job Title_Technical S

In [ ]:
# 각 국가별 직종별 종사자 수 합계
total_job_title_counts = pd.DataFrame(country_job_title_counts).sum(axis=1)

# 합계가 0인 직군 제거
filtered_job_titles = total_job_title_counts[total_job_title_counts > 0 ]

# 필터링된 직군 종사자 수 출력
print(filtered_job_titles.sort_values(ascending=False))

Job Title_Software Engineer               809.0
Job Title_Data Scientist                  515.0
Job Title_Data Analyst                    391.0
Job Title_Software Engineer Manager       376.0
Job Title_Project Engineer                317.0
Job Title_Full Stack Engineer             304.0
Job Title_Front end Developer             270.0
Job Title_Back end Developer              242.0
Job Title_Software Developer              186.0
Job Title_Web Developer                   129.0
Job Title_Director of Data Science         57.0
Job Title_Data Engineer                     4.0
Job Title_Engineer                          2.0
Job Title_IT Consultant                     2.0
Job Title_IT Support Specialist             2.0
Job Title_Director of Engineering           2.0
Job Title_Developer                         1.0
Job Title_Web Designer                      1.0
Job Title_Technical Writer                  1.0
Job Title_Technical Support Specialist      1.0
Job Title_Software Project Manager      

In [ ]:
# 각 'Job Title' 열에 대한 표본 수 계산
job_title_sample_counts = combined_data[job_title_columns].sum()

# 표본 수가 10개 미만인 'Job Title' 열 식별
low_sample_job_titles = job_title_sample_counts[job_title_sample_counts < 10].index

# 해당 열에서 값이 1인 행 제거
for job_title_col in low_sample_job_titles:
    combined_data = combined_data[combined_data[job_title_col] == 0]

# 제거 후 행의 개수 확인
print(f"Number of rows after removal: {len(combined_data)}")

Number of rows after removal: 3596


In [ ]:
# 각 직업의 표본 수 계산
job_title_sample_counts = combined_data[job_title_columns].sum(axis=0)

# 표본 수가 10개 미만인 직업 확인
low_sample_job_titles = job_title_sample_counts[job_title_sample_counts < 10].index

# 이러한 직업을 가진 행들 제거
for job_title_col in low_sample_job_titles:
    combined_data = combined_data[combined_data[job_title_col] == 0]

# 훈련 및 테스트 세트 분할
X = combined_data.drop('Salary', axis=1)
y = combined_data['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 재훈련 및 평가
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred))
r2_rf = r2_score(y_test, y_pred)


#랜덤 포레스트 평가지표
print("===랜덤포레스트===")
print(f"RMSE: {rmse_rf}")
print(f"R² score: {r2_rf}")
print("\n")

# 선형 회귀 모델 생성 및 학습
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 테스트 세트로 예측
y_pred = lr_model.predict(X_test)

# 선형회귀 평가
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred))
r2_linear = r2_score(y_test, y_pred)

# 결과 출력
print("===선형회귀===")
print(f"RMSE: {rmse_linear}")
print(f"R² score: {r2_linear}")

===랜덤포레스트===
RMSE: 9886.995779135794
R² score: 0.9608941684195568


===선형회귀===
RMSE: 27066.961332001487
R² score: 0.7069161869953919


In [ ]:
# 각 직군별 종사자 수 계산
job_title_counts = combined_data[job_title_columns].sum()

# 각 직군별 종사자 수 계산(유의미한 값들만)
job_title_counts = job_title_counts[job_title_counts > 10]

# 결과 출력
print(job_title_counts.sort_values(ascending=False))

Job Title_Software Engineer            809.0
Job Title_Data Scientist               515.0
Job Title_Data Analyst                 391.0
Job Title_Software Engineer Manager    376.0
Job Title_Project Engineer             317.0
Job Title_Full Stack Engineer          304.0
Job Title_Front end Developer          270.0
Job Title_Back end Developer           242.0
Job Title_Software Developer           186.0
Job Title_Web Developer                129.0
Job Title_Director of Data Science      57.0
dtype: float64


30살, 5년 경력, 학사, 미국 기업

In [ ]:
# 예측을 위한 공통 조건 설정
common_conditions = {
    'Age': 30,
    'Years of Experience': 5,
    'Education Level_1': 1,  # 학사 학위
    'Country_USA': 1
}

# 비교할 직군
job_titles = ['Software Engineer', 'Full Stack Engineer', 'Data Scientist', 'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer', 'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science']
# 각 직군별 예측 연봉 계산
predicted_salaries = {}
for job_title in job_titles:
    # 원-핫 인코딩된 데이터셋에서 입력 행 생성
    input_data = np.zeros(len(X.columns))
    for col, value in common_conditions.items():
        input_data[X.columns.get_loc(col)] = value
    input_data[X.columns.get_loc(f'Job Title_{job_title}')] = 1

    # 연봉 예측
    predicted_salary = rf_model.predict([input_data])[0]
    predicted_salaries[job_title] = predicted_salary

predicted_salaries

{'Software Engineer': 81642.0,
 'Full Stack Engineer': 102191.25916666666,
 'Data Scientist': 137766.66666666666,
 'Software Engineer Manager': 92039.38916666666,
 'Data Analyst': 150600.0,
 'Project Engineer': 91914.73333333332,
 'Back end Developer': 98897.7225,
 'Front end Developer': 96177.34666666666,
 'Software Developer': 77633.67,
 'Web Developer': 70200.0,
 'Director of Data Science': 92039.38916666666}

지금의 내가 만약 미국 기업에 취직한다면?(학력 수준을 바꿔도 다르게 안나옴)

In [ ]:
# 예측을 위한 공통 조건 설정
common_conditions = {
    'Age': 26,
    'Years of Experience': 1,
    'Education Level_1': 1,  # 학사 학위
    'Country_USA': 1
}

# 비교할 직군
job_titles = ['Software Engineer', 'Full Stack Engineer', 'Data Scientist', 'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer', 'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science']
'Software Engineer', 'Full Stack Engineer', 'Data Scientist',
'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer',
'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science'
# 각 직군별 예측 연봉 계산
predicted_salaries = {}
for job_title in job_titles:
    # 원-핫 인코딩된 데이터셋에서 입력 행 생성
    input_data = np.zeros(len(X.columns))
    for col, value in common_conditions.items():
        input_data[X.columns.get_loc(col)] = value
    input_data[X.columns.get_loc(f'Job Title_{job_title}')] = 1

    # 연봉 예측
    predicted_salary = rf_model.predict([input_data])[0]
    predicted_salaries[job_title] = predicted_salary

predicted_salaries

{'Software Engineer': 50067.5,
 'Full Stack Engineer': 50664.81,
 'Data Scientist': 103436.81,
 'Software Engineer Manager': 62439.31666666666,
 'Data Analyst': 107619.10714285714,
 'Project Engineer': 50755.36,
 'Back end Developer': 53462.79,
 'Front end Developer': 55682.5,
 'Software Developer': 39490.0,
 'Web Developer': 40955.0,
 'Director of Data Science': 50664.81}

In [ ]:
# 예측을 위한 공통 조건 설정
common_conditions = {
    'Age': 26,
    'Years of Experience': 1,
    'Education Level_1': 3,  # 박사 학위
    'Country_USA': 1
}

# 비교할 직군
job_titles = ['Software Engineer', 'Full Stack Engineer', 'Data Scientist', 'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer', 'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science']
'Software Engineer', 'Full Stack Engineer', 'Data Scientist',
'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer',
'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science'
# 각 직군별 예측 연봉 계산
predicted_salaries = {}
for job_title in job_titles:
    # 원-핫 인코딩된 데이터셋에서 입력 행 생성
    input_data = np.zeros(len(X.columns))
    for col, value in common_conditions.items():
        input_data[X.columns.get_loc(col)] = value
    input_data[X.columns.get_loc(f'Job Title_{job_title}')] = 1

    # 연봉 예측
    predicted_salary = rf_model.predict([input_data])[0]
    predicted_salaries[job_title] = predicted_salary

predicted_salaries

{'Software Engineer': 50067.5,
 'Full Stack Engineer': 50664.81,
 'Data Scientist': 103436.81,
 'Software Engineer Manager': 62439.31666666666,
 'Data Analyst': 107619.10714285714,
 'Project Engineer': 50755.36,
 'Back end Developer': 53462.79,
 'Front end Developer': 55682.5,
 'Software Developer': 39490.0,
 'Web Developer': 40955.0,
 'Director of Data Science': 50664.81}

경력이 쌓인다면?

In [ ]:
# 예측을 위한 공통 조건 설정
common_conditions = {
    'Age': 30,
    'Years of Experience': 3,
    'Education Level_1': 1,  # 학사 학위
    'Country_USA': 1
}

# 비교할 직군

# job_titles = ['Developer', 'Data Scientist', 'Project Engineer', 'Back end Developer', 'Front end Developer', 'Full Stack Engineer']
job_titles = ['Software Engineer', 'Full Stack Engineer', 'Data Scientist', 'Software Engineer Manager', 'Data Analyst', 'Project Engineer', 'Back end Developer', 'Front end Developer', 'Software Developer', 'Web Developer', 'Director of Data Science']

# 각 직군별 예측 연봉 계산
predicted_salaries = {}
for job_title in job_titles:
    # 원-핫 인코딩된 데이터셋에서 입력 행 생성
    input_data = np.zeros(len(X.columns))
    for col, value in common_conditions.items():
        input_data[X.columns.get_loc(col)] = value
    input_data[X.columns.get_loc(f'Job Title_{job_title}')] = 1

    # 연봉 예측
    predicted_salary = rf_model.predict([input_data])[0]
    predicted_salaries[job_title] = predicted_salary

predicted_salaries

{'Software Engineer': 75302.71761904762,
 'Full Stack Engineer': 86324.5042857143,
 'Data Scientist': 100021.36666666668,
 'Software Engineer Manager': 76512.66095238095,
 'Data Analyst': 63200.0,
 'Project Engineer': 74123.38333333333,
 'Back end Developer': 83745.76095238097,
 'Front end Developer': 78097.94,
 'Software Developer': 63460.0,
 'Web Developer': 59900.0,
 'Director of Data Science': 76512.66095238095}